In [1]:
import streamlit as st
from openai import OpenAI
from dotenv import load_dotenv
import os
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [11]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [12]:
model = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo", temperature=0.2)
parser = StrOutputParser()

In [14]:
file_loader = PyPDFLoader('Hitachi Manual.pdf')
pages = file_loader.load_and_split()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_pages = splitter.split_documents(pages)
vector_storage = FAISS.from_documents(split_pages, OpenAIEmbeddings())
retriever = vector_storage.as_retriever()
question_template = """
You're a smart chatbot that answers user generated prompts only based on the context given to you.
You don't make any assumptions.
context:{context}
question:{question}
"""
prompt = PromptTemplate.from_template(template=question_template)